In [ ]:
import yasp
# yasp.debug = True
import sys
import os
yasp_prefix = yasp.yasp_feature("prefix")
site_packages = os.path.join(sys.prefix, 'lib', f'python{sys.version[:3]}', 'site-packages')
add_path = os.path.join(yasp_prefix, 'lib', f'python{sys.version[:3]}', 'site-packages')
sys.path.append(add_path)

In [ ]:
yasp.module_load_cppyy('bundle/hepbase')
yasp.module_load_cppyy('heppyy/current')
yasp.module_load_cppyy('alian/current')

import heppyy
fj = heppyy.load_cppyy('fastjet')
std = heppyy.load_cppyy('std')
alian_cpp = heppyy.load_cppyy('alian')

In [ ]:
import alian
if yasp.in_jupyter_notebook():
  from tqdm.notebook import tqdm
else:
  from tqdm import tqdm


In [ ]:
fj.ClusterSequence().print_banner()

jet_algorithm = fj.antikt_algorithm
jet_R = 0.6
jet_eta_max = 2.0
bg_y_max = 0.9
bg_grid_spacing = 0.1

jet_pt_min = 100
jet_def = fj.JetDefinition(jet_algorithm, jet_R)
area_def = fj.AreaDefinition(fj.active_area, fj.GhostedAreaSpec(jet_eta_max + jet_R, 1, 0.01))
jet_selector = fj.SelectorAbsEtaMax(jet_eta_max) * fj.SelectorPtMin(jet_pt_min)

# need for a background estimator?
bg_estimator = fj.GridMedianBackgroundEstimator(bg_y_max, bg_grid_spacing)

jet_def_lund = fj.JetDefinition(fj.cambridge_algorithm, 1.0)
lund_gen = fj.contrib.LundGenerator(jet_def_lund)
print('making lund diagram for all jets...')
print(f' {lund_gen.description()}')


In [ ]:
pt_hat_min = jet_pt_min / (jet_R * 2.)

pythia_cmnd = os.path.join(alian.alian_settings.src_path, 'config/pythia-pp-hardQCD-5TeV-Monash.cmnd')
pythia_settings = ['PhaseSpace:pThatMin = {}'.format(pt_hat_min)]

from heppyy.pythia_util.configuration import create_and_init_pythia
pythia = create_and_init_pythia(pythia_settings, pythia_cmnd)

In [ ]:
from alian.io.pythia_io import psj_from_particle_with_index
pythia_offset_index = 0
nevents = 10
i = 0
for i in tqdm(range(nevents)):
    if not pythia.next():
        continue
    print(f'Event: {i} nparts: {pythia.event.size()}')
    i += 1
    psjv = std.vector[fj.PseudoJet]([psj_from_particle_with_index(p, i + pythia_offset_index) 
                                    for i, p in enumerate(pythia.event) if p.isFinal() and p.isVisible() and p.isCharged()])
    ca = fj.ClusterSequenceArea(psjv, jet_def, area_def)
    jets = fj.sorted_by_pt(jet_selector(ca.inclusive_jets()))
    leadpt = -1
    if jets.size() > 0:
        leadpt = jets[0].perp()
    else:
        continue
    print(f'{i} : n particles: {psjv.size()} -> njets: {jets.size()} | highest pt: {leadpt}')
    for j in jets:
        lunds = lund_gen.result(j)
        print(f'jet pT={j.perp()}')
        for i, l in enumerate(lunds):
            print('- L {} pT={:5.2f} eta={:5.2f}'.format(i, l.pair().perp(), l.pair().eta()))
            print('  Deltas={}'.format(l.Delta()))
            print('  kts={}'.format(l.kt()))
            print()


In [ ]:
pythia_offset_index = 0
nevents = 10000
i = 0
all_jets = []
for i in tqdm(range(nevents)):
    if not pythia.next():
        continue
    # print(f'Event: {i} nparts: {pythia.event.size()}')
    i += 1
    psjv = std.vector[fj.PseudoJet]([psj_from_particle_with_index(p, i + pythia_offset_index) 
                                    for i, p in enumerate(pythia.event) if p.isFinal() and p.isVisible() and p.isCharged()])
    ca = fj.ClusterSequenceArea(psjv, jet_def, area_def)
    jets = fj.sorted_by_pt(jet_selector(ca.inclusive_jets()))
    leadpt = -1
    if jets.size() > 0:
        leadpt = jets[0].perp()
    else:
        continue
    # print(f'{i} : n particles: {psjv.size()} -> njets: {jets.size()} | highest pt: {leadpt}')
    for j in jets:
        lunds = lund_gen.result(j)
        # print(f'jet pT={j.perp()}')
        j_dict = {'pt': j.perp(), 'eta': j.eta(), 'phi': j.phi(), 'area': j.area(), 'lunds': []}
        for i, l in enumerate(lunds):
            pt1 = l.harder().perp()
            pt2 = l.softer().perp()
            lund_dict = {'pt': l.pair().perp(), 'pt1': pt1, 'pt2': pt2, 'eta': l.pair().eta(), 'kt': l.kt(), 'delta': l.Delta()}
            j_dict['lunds'].append(lund_dict)
        all_jets.append(j_dict)
        
print(f'found {len(all_jets)} jets')
# print(all_jets)

In [ ]:
import pandas as pd
import numpy as np

records = []
for jet in all_jets:
    lunds = jet['lunds']
    for i in range(len(lunds) - 1):
        kt_i, delta_i = lunds[i]['kt'], lunds[i]['delta']
        kt_j, delta_j = lunds[i+1]['kt'], lunds[i+1]['delta']
        x_i, y_i = np.log(1/delta_i), np.log(kt_i)
        x_j, y_j = np.log(1/delta_j), np.log(kt_j)
        records.append({'x': x_i, 'y': y_i, 'dx': x_j - x_i, 'dy': y_j - y_i})

df = pd.DataFrame(records)

# Define grid and bin the (x,y) positions
nx, ny = 25, 25
x_bins = np.linspace(df['x'].min(), df['x'].max(), nx + 1)
y_bins = np.linspace(df['y'].min(), df['y'].max(), ny + 1)
df['ix'] = pd.cut(df['x'], bins=x_bins, labels=False, include_lowest=True)
df['iy'] = pd.cut(df['y'], bins=y_bins, labels=False, include_lowest=True)

# Compute average displacement per bin
group = df.groupby(['ix', 'iy']).agg(
    x=('x', 'mean'),
    y=('y', 'mean'),
    dx=('dx', 'mean'),
    dy=('dy', 'mean')
).dropna().reset_index()

import matplotlib.pyplot as plt
# Plot the trajectory current
plt.figure()
plt.quiver(group['x'], group['y'], group['dx'], group['dy'])
plt.xlabel(r'ln(1/$delta$)')
plt.ylabel('ln(kT)')
plt.title(f'Trajectory Current (Example with {len(all_jets)} Jets with pt > {jet_pt_min} GeV)')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
import matplotlib.colors as mcolors

# Grid resolution
nx, ny = 20, 20
x_bins = np.linspace(df['x'].min(), df['x'].max(), nx + 1)
y_bins = np.linspace(df['y'].min(), df['y'].max(), ny + 1)

# Assign each point to a bin
df['ix'] = pd.cut(df['x'], bins=x_bins, labels=False, include_lowest=True)
df['iy'] = pd.cut(df['y'], bins=y_bins, labels=False, include_lowest=True)

# Compute average displacement per bin
group = (
    df.groupby(['ix', 'iy'])
      .agg(x=('x', 'mean'), y=('y', 'mean'),
           dx=('dx', 'mean'), dy=('dy', 'mean'))
      .dropna()
      .reset_index()
)

# Plot
plt.figure(figsize=(8, 6))

# 1) density background
# Normalize by number of jets
weights = np.ones_like(df['x']) / len(all_jets)

plt.hist2d(
    df['x'], df['y'],
    bins=[x_bins, y_bins],
    cmap='Blues',
    norm=mcolors.LogNorm(),
    weights=weights
)
plt.clim(0.001, 1.0)
plt.colorbar(label='Splitting count per jet (log scale)')

# 2) trajectory current
plt.quiver(group['x'], group['y'], group['dx'], group['dy'], 
           angles='xy', scale_units='xy', scale=2.0, width=0.003, color='red', alpha=0.5)

plt.xlabel('ln(1/δ)')
plt.ylabel('ln(kT)')
plt.title(f'Splitting Density with Trajectory Current - {len(all_jets)} Jets with pt > {jet_pt_min} GeV')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Compute the density (weight) for each bin
hist, xedges, yedges = np.histogram2d(df['x'], df['y'], bins=[x_bins, y_bins], weights=weights)

# Map group['ix'], group['iy'] to the corresponding density
arrow_weights = hist[group['ix'], group['iy']]

# Optionally, normalize or rescale arrow_weights for better visualization
arrow_weights = arrow_weights / arrow_weights.max()  # scale to [0, 1]

# Scale dx and dy by the weights
dx_scaled = group['dx'] * arrow_weights
dy_scaled = group['dy'] * arrow_weights

plt.figure(figsize=(8, 6))
plt.hist2d(
    df['x'], df['y'],
    bins=[x_bins, y_bins],
    cmap='Blues',
    norm=mcolors.LogNorm(),
    weights=weights
)
plt.clim(0.001, 1.0)
plt.colorbar(label='Splitting count per jet (log scale)')

# Quiver with scaled arrows
plt.quiver(group['x'], group['y'], dx_scaled, dy_scaled, 
           angles='xy', scale_units='xy', scale=0.7, width=0.003, color='red', alpha=0.5)

plt.xlabel('ln(1/δ)')
plt.ylabel('ln(kT)')
plt.title(f'Splitting Density with Weighted Trajectory Current - {len(all_jets)} Jets with pt > {jet_pt_min} GeV')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.colors as mcolors

# Grid resolution
nx, ny = 20, 20
x_bins = np.linspace(df['x'].min(), df['x'].max(), nx + 1)
y_bins = np.linspace(df['y'].min(), df['y'].max(), ny + 1)

# Assign each point to a bin
df['ix'] = pd.cut(df['x'], bins=x_bins, labels=False, include_lowest=True)
df['iy'] = pd.cut(df['y'], bins=y_bins, labels=False, include_lowest=True)

# Compute average displacement per bin (group is calculated on full df)
group = (
    df.groupby(['ix', 'iy'])
      .agg(x=('x', 'mean'), y=('y', 'mean'),
           dx=('dx', 'mean'), dy=('dy', 'mean'))
      .dropna()
      .reset_index()
)

# Apply cut for plotting only (ln(kT) > -4)
df_plot = df[df['y'] > -4]

plt.figure(figsize=(8, 6))

# 1) density background (only for ln(kT) > -4)
weights_plot = np.ones_like(df_plot['x']) / len(all_jets)
plt.hist2d(
    df_plot['x'], df_plot['y'],
    bins=[x_bins, y_bins],
    cmap='Blues',
    norm=mcolors.LogNorm(),
    weights=weights_plot
)
plt.clim(0.001, 1.0)
plt.colorbar(label='Splitting count per jet (log scale)')

# 2) trajectory current (quiver from full group, not cut)
plt.quiver(group['x'], group['y'], group['dx'], group['dy'], 
           angles='xy', scale_units='xy', scale=2.0, width=0.003, color='red', alpha=0.5)

plt.xlabel('ln(1/δ)')
plt.ylabel('ln(kT)')
plt.title(f'Splitting Density with Trajectory Current - {len(all_jets)} Jets with pt > {jet_pt_min} GeV')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.colors as mcolors

# Grid resolution
nx, ny = 20, 20
x_bins = np.linspace(df['x'].min(), df['x'].max(), nx + 1)
y_bins = np.linspace(df['y'].min(), df['y'].max(), ny + 1)

# Assign each point to a bin
df['ix'] = pd.cut(df['x'], bins=x_bins, labels=False, include_lowest=True)
df['iy'] = pd.cut(df['y'], bins=y_bins, labels=False, include_lowest=True)

# Compute average displacement per bin (group is calculated on full df)
group = (
    df.groupby(['ix', 'iy'])
      .agg(x=('x', 'mean'), y=('y', 'mean'),
           dx=('dx', 'mean'), dy=('dy', 'mean'))
      .dropna()
      .reset_index()
)

# Apply cut for plotting only (ln(kT) > -4)
df_plot = df[df['y'] > -4]

# Drop arrows starting at ln(kT) < -4
group_plot = group[group['y'] > -4]

plt.figure(figsize=(8, 6))

# 1) density background (only for ln(kT) > -4)
weights_plot = np.ones_like(df_plot['x']) / len(all_jets)
plt.hist2d(
    df_plot['x'], df_plot['y'],
    bins=[x_bins, y_bins],
    cmap='Blues',
    norm=mcolors.LogNorm(),
    weights=weights_plot
)
plt.clim(0.001, 1.0)
plt.colorbar(label='Splitting count per jet (log scale)')

# 2) trajectory current (quiver only for arrows starting at ln(kT) > -4)
plt.quiver(group_plot['x'], group_plot['y'], group_plot['dx'], group_plot['dy'], 
           angles='xy', scale_units='xy', scale=5, width=0.003, color='blue', alpha=1.0)

plt.xlabel('ln(1/δ)')
plt.ylabel('ln(kT)')
# plt.ylim(-4, 4)  # <-- Adjust y-axis: show only ln(kT) > -4
plt.ylim(-4, None)  # <-- Adjust y-axis: show only ln(kT) > -4
plt.title(f'Splitting Density with Trajectory Current - {len(all_jets)} Jets with pt > {jet_pt_min} GeV')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()